In [2]:
import helpers as h

import numpy as np
import pandas as pd
from scipy.optimize import minimize, NonlinearConstraint, BFGS, Bounds

from icecream import ic

In [3]:
# constants 
# states_path = "data/states/nga_admbnda_adm1_osgof_20161215.shp"
# states = gpd.read_file(states_path)

state_areas = pd.read_csv(f"clean_data/state_areas.csv")
data_path_names = ["wind_speed", "temperature", "solar_flux"]

data_averages = {}
for name in data_path_names:
    data_averages[name] = pd.read_csv(f"clean_data/{name}.csv", index_col=0)

avg_vals = {n: i["value"] for n, i in data_averages.items()}

In [4]:
def get_constants():
    data_path_names = ["wind_speed", "temperature", "solar_flux"]

    data_averages = {}
    for name in data_path_names:
        data_averages[name] = pd.read_csv(f"clean_data/{name}.csv", index_col=0)

    avg_vals = {n: i["value"] for n, i in data_averages.items()}

    state_areas = pd.read_csv(f"clean_data/state_areas.csv")

    F_cur=avg_vals["solar_flux"]
    T_a=avg_vals["temperature"]
    w=avg_vals["wind_speed"]

    return (F_cur, T_a, w, state_areas)

    

In [5]:
class optimize_energy():
    def __init__(self):
        c = get_constants()
        self.F_cur = c[0]
        self.T_a = c[1]
        self.w = c[2] 
        self.state_areas = c[3]

    def do_calc(self, x):
        """ 
        - pl_solar, pl_wind ~ percent of land 
        - ns_solar, ns_wind ~ number of states 
        """
        self.pl_solar, self.pl_wind, self.ns_solar, self.ns_wind = x

        # capacities for each state 
        P_ac = h.calculate_power_panel(self.F_cur, self.T_a, self.w)
        P_m, P_t, CF, P_t_after_losses, P_r = h.calculate_power_turbine(V_m=self.w)
        self.P_ac = P_ac
        self.P_t_after_losses =  P_t_after_losses

        # distributions
        solar_res = h.calc_num_panels(self.pl_solar, 
            self.ns_wind, 
            self.state_areas, 
            P_ac)
        wind_res = h.calc_num_turbines(self.pl_wind, 
            self.ns_wind, 
            self.state_areas, 
            P_t_after_losses, 
            P_r)

        self.solar_res = solar_res
        self.wind_res = wind_res 
        

        return solar_res, wind_res

    def min_devices(self, x):
        solar_res, wind_res = self.do_calc(x)
        self.num_devices = solar_res["n_panels"].sum() + wind_res["n_turbines"].sum()

        return self.num_devices

    def meet_energy(self, x):
        solar_res, wind_res = self.do_calc(x)
        self.total_energy = solar_res["total_energy (mwh)"] + wind_res["total_energy (mwh)"]
        self.energy_need = 26_5337_353 # 27 million MWh 
        self.dif = self.total_energy - self.energy_need # > 0 

        return self.dif 

    def show_results(self):
        wind_diam = 5 # m 
        solar_area = 1.5 # m
        wind_data = {
            "num_devices": self.wind_res["n_turbines"].sum(),
            "installed_power": self.wind_res["installed_power"].sum(),
            "num_states": self.ns_wind,
            "spacing": self.wind_res["land_avail"].sum(),
            "footprint": self.wind_res["n_turbines"].sum()*wind_diam
        }
        solar_data = {
            "num_devices": self.solar_res["n_panels"].sum(),
            "installed_power": self.solar_res["true_power"].sum(),
            "num_states": self.ns_solar,
            "spacing": self.solar_res["land_avail"].sum(),
            "footprint": self.solar_res["n_panels"].sum()*solar_area
        }

        sum_data = {
            "total_energy": self.solar_res["total_energy (mwh)"] + self.wind_res["total_energy (mwh)"],
            "total_footprint": wind_data["footprint"] + solar_data["footprint"],
            "percentage land": 100*(wind_data["footprint"] + solar_data["footprint"]) / self.state_areas["area (m2)"].sum()
        }

        self.devices_df  = pd.DataFrame([solar_data,wind_data], index=["Solar","Wind"], )
        self.devices_df.columns = ["N. Devices", "Possible Power (MW)", "N. States", "Spacing (m2)", "Footprint (m2)"]


        self.total_df =  pd.DataFrame(sum_data, index=[0])
        self.total_df.columns = ["Total Energy (MWh)", "Total Footprint (m2)", "% Land"]

        return self.total_df
    

In [6]:
x0 = [0.02, 0.02, 4, 4]
o = optimize_energy()
o.min_devices(x0)
o.meet_energy(x0)

19582530264.701317

In [7]:
o.show_results()

,Total Energy (MWh),Total Footprint (m2),% Land
0,1.984787e+10,3.290548e+09,0.360929


In [8]:
o.devices_df

,N. Devices,Possible Power (MW),N. States,Spacing (m2),Footprint (m2)
Solar,2.193651e+09,2.253846e+12,4,3.290476e+09,3.290476e+09
Wind,1.437500e+04,5.333035e+10,4,2.693342e+09,7.187500e+04


## do optimization

In [9]:
o = optimize_energy()
x0 = [0.01, 0.01, 2, 2]

# bounds = Bounds([0, 1], [0, 1], [0, 35],[0,35] )
# bounds = Bounds([0,0, 0, 0], [1,1, 35, 35])
nlc  = NonlinearConstraint(fun=o.meet_energy, lb=0, ub=np.inf, jac='2-point', hess=BFGS())
res = minimize(fun=o.min_devices, x0=x0, constraints=nlc, method='trust-constr', options={'disp': True, 'maxiter': 300}, )
# bounds=bounds

# changes => np.inf + constraints that none of the xs can be < 0 

`gtol` termination condition is satisfied.
Number of iterations: 4, function evaluations: 20, CG iterations: 3, optimality: 0.00e+00, constraint violation: 0.00e+00, execution time: 0.063 s.


/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '
/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the f

In [10]:
o.show_results()

,Total Energy (MWh),Total Footprint (m2),% Land
0,2.653387e+08,40361794.0,0.004427


In [11]:
o.devices_df

,N. Devices,Possible Power (MW),N. States,Spacing (m2),Footprint (m2)
Solar,26898166.0,2.781969e+10,2.0,4.034725e+07,40347249.0
Wind,2909.0,1.113052e+10,2.0,5.621551e+08,14545.0


In [72]:
o.dif

3955.8106880784035

In [ ]:
res.x

array([1.63255237e-04, 2.00000000e-02, 4.00000000e+00, 4.00000000e+00])

In [ ]:
o = optimize_energy()
x0 = [0.1, 0.1, 30, 30]

nlc  = NonlinearConstraint(fun=o.meet_energy, lb=0, ub=1e3, jac='2-point', hess=BFGS())
res = minimize(fun=o.min_devices, x0=x0, constraints=nlc, method='trust-constr', options={'disp': True, 'maxiter': 300})

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


`xtol` termination condition is satisfied.
Number of iterations: 284, function evaluations: 2610, CG iterations: 316, optimality: 1.22e-04, constraint violation: 3.32e+00, execution time: 1.6e+01 s.


In [ ]:
res.x

array([-6.92195248e-04,  1.00000000e-01,  3.00000000e+01,  3.00000000e+01])

In [ ]:
o.dif

-3.319736957550049

In [ ]:
o = optimize_energy()
x0 = [0.3, 0.4, 10, 30]

nlc  = NonlinearConstraint(fun=o.meet_energy, lb=0, ub=0, jac='2-point', hess=BFGS())
res = minimize(fun=o.min_devices, x0=x0, constraints=nlc, method='trust-constr', options={'disp': True, 'maxiter': 300})

/Users/julietnwagwuume-ezeoke/_ClassCode/classenv/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


`xtol` termination condition is satisfied.
Number of iterations: 19, function evaluations: 70, CG iterations: 18, optimality: 0.00e+00, constraint violation: 7.03e+00, execution time: 0.45 s.


In [ ]:
res.x

array([-1.00493545e-02,  1.33735405e+00,  1.00000000e+01,  3.00000000e+01])

- things to minimize potentially
  - total land used 
  - number of turbines + number of panels  ~ cost 
- start by definining seperate function 

- [mnimize where output has contraints](https://stackoverflow.com/questions/64169852/optimizing-input-where-output-has-constraints-in-python)
- [use a class for constants](https://stackoverflow.com/questions/64205381/can-i-send-arguments-to-a-constraint-function-using-scipy-optimize-nonlinearcons)

- wind 
    - num turbines
    - installed power 
    - total energy to produced
    - number of states 
    - spacing => land area 
    - footprint 
    - total land area 
- solar
    - num panels 
    - installed power 
    - total energy to produced
    - number of states 
    - spacing => land area 
    - footprint 
    - total land area 
- sum energy 
- sum land 
- land percent (footprint)

In [ ]:
o.dif

-10.796165466308594

In [ ]:
o.num_devices

-5623709918.0

In [ ]:
# things to minimize potentially 
    